# Agent Loan Decisioning

In [1]:
import datetime
from datetime import date,timedelta
import numpy as np
import pandas as pd

1.	Agent Length on Service
2.	Agent Frequency of use
3.	Agent Frequency of rebalance
4.	Median Cash-In for last 90 days
5.	Mode Cash-Ins for last 90 days
6.	Median Cash-Out for last 90 days
7.	Mode Cash-Outs for last 90 days


In [2]:
#Reading data from csv file

momo_data = pd.read_csv('/Cleaned_momo.csv', skipinitialspace = True)
momo_data.head()

,TRANSACTION_DATE,TRANSACTION_TIME,FROM_ACCT,FROM_NAME,FROM_NO,TRANS_TYPE,AMOUNT,FEES,BA_ BEFORE,BAL_AFTER,TO_NO,TO_NAME,TO_ACCT,F_ID,REF,OVA
0,1/31/22,4:12:00,43122976,Emmanuel Lamptey,244471083,PAYMENT,40.0,0.5,1741.08,1700.58,243868392,INTEROPERABILIT Y PUSH OVA,54814522.0,15865181460,"Prince Abroquah,2332082 09124,1",mmipush
1,1/31/22,11:53:00,43122976,Emmanuel Lamptey,244471083,DEBIT,10.0,0.0,1751.08,1741.08,0,cis,53169694.0,15861507663,Internet Bundle,cis
2,1/28/22,1:51:00,43122976,Emmanuel Lamptey,244471083,TRANSFER,20.0,0.0,1771.08,1751.08,552384736,nancy akuffo,53815211.0,15824193330,1,Internal
3,1/27/22,2:55:00,43122976,Emmanuel Lamptey,244471083,TRANSFER,1680.0,10.0,3461.08,1771.08,557938985,PATRICIA ANNOR,52297015.0,15810636337,1,Internal
4,1/27/22,2:53:00,53104390,FIDELITY BANK LTD BREKUM BRANCH,549293662,CASH_IN,1690.0,0.0,1771.08,3461.08,244471083,Emmanuel Lamptey,43122976.0,15810617557,NaN,Internal


In [3]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TRANSACTION_DATE  89 non-null     object 
 1   TRANSACTION_TIME  89 non-null     object 
 2   FROM_ACCT         89 non-null     int64  
 3   FROM_NAME         89 non-null     object 
 4   FROM_NO           89 non-null     int64  
 5   TRANS_TYPE        89 non-null     object 
 6   AMOUNT            89 non-null     float64
 7   FEES              89 non-null     float64
 8   BA_ BEFORE        89 non-null     float64
 9   BAL_AFTER         89 non-null     float64
 10  TO_NO             89 non-null     int64  
 11  TO_NAME           87 non-null     object 
 12  TO_ACCT           87 non-null     float64
 13  F_ID              89 non-null     int64  
 14  REF               70 non-null     object 
 15  OVA               89 non-null     object 
dtypes: float64(5), int64(4), object(7)
memory usag

In [4]:
momo_data = momo_data.rename(columns=lambda x: x.strip())

In [5]:
momo_data = momo_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [6]:
masterData = []
userData= {}

In [7]:
# 2. Agent Frequency of use

# momo_data.count_rows()
rows = momo_data.count()[0]
rows
userData["freq_of_active"] = rows
userData["freq_of_active"]

89

In [8]:
# 3.	Agent Frequency of rebalance

MOMO=momo_data[(momo_data == 'CASH_IN').any(axis=1)]
MOMOR = MOMO.count()[0]
userData["freq_of_repl"] = MOMOR
userData["freq_of_repl"]


2

In [9]:
# 4.	Median Cash-In for last 90 days

# df.loc[df['B'] == 3, 'A']
meanie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].mean()
meanie
userData["mean_cash_in"] = meanie
userData["mean_cash_in"]

1047.5

In [10]:
userData["mean_cash_in_calc"] = meanie*4

In [11]:
# 4.	Median Cash-Out for last 90 days

# df.loc[df['B'] == 3, 'A']
meaniee = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].mean()
meaniee
userData["mean_cash_out"] = meaniee
userData["mean_cash_out"]

400.0

In [12]:
userData["mean_cash_out_calc"] = meaniee*4

In [13]:
user_copy = userData.copy()
masterData.append(user_copy)

In [14]:
### scoring

inelligible = []
elligible = []

In [15]:
Credit_score = 0

## Frequency of use

for index in range(len(masterData)):
    
    if masterData[index]["freq_of_active"]> 65 and masterData[index]["freq_of_active"] <= 75:
        user_copy = masterData[index]
#         print(user_copy)
        Credit_score += 50 
    elif masterData[index]["freq_of_active"]> 75:
        elig_copy = masterData[index]
        Credit_score+= 100

#         print(elig_copy)
#     elligible.append(elig_copy)
        
        
    if masterData[index]["freq_of_repl"]>= 45 and masterData[index]["freq_of_repl"] <= 54:
        user_copy = masterData[index]
#         print(user_copy)
        Credit_score += 50 
    elif masterData[index]["freq_of_active"]> 54:
        elig_copy = masterData[index]
        Credit_score+= 100

#         print(elig_copy)
#     elligible.append(elig_copy)
        
        
        #median cash-in
        
    if masterData[index]["mean_cash_in_calc"] > 5000:
        user_copy = masterData[index]
#         print(user_copy)
        Credit_score += 50
        
    else:
        Credit_score+= 0

#         print(elig_copy)
#         elligible.append(elig_copy)
        
          #median cash-out
        
    if masterData[index]["mean_cash_out_calc"] > 5000:
        user_copy = masterData[index]
#         print(user_copy)
        Credit_score += 50 
        
    else:
        elig_copy = masterData[index]
        Credit_score+= 0

#         print(elig_copy)
elligible.append(elig_copy)
        
        
        
        
print("Inelligble ", inelligible)
print('\n')
print("Elligble ", elligible)
    
    
        

Inelligble  []


Elligble  [{'freq_of_active': 89, 'freq_of_repl': 2, 'mean_cash_in': 1047.5, 'mean_cash_in_calc': 4190.0, 'mean_cash_out': 400.0, 'mean_cash_out_calc': 1600.0}]


In [ ]:
masterData

In [ ]:
Credit_score

In [ ]:

# 2.	Agent Frequency of use

# With Loan Input
# easier to do 3x calculation

print("Checking loan elligibility...\n")




userData["firstname"]=input("What is your first name: ").lower()

userData["lastname"]=input("What is your last name: ").lower()

    
while True:
    try:

        birthday=input("What is your birthday? (in DD/MM/YYYY): ")

        birthdate = date_convert((birthday))
        birthdate=datetime.datetime.strptime(birthday,"%d/%m/%Y").date()
        userData["age"]=age((birthdate))
        break
    except ValueError:
        print("\n")
        print("Oops!  Date format is wrong.  Try again in correct format.")


while True:
    married = input('\nAre you married?\n\nA) Yes \nB) No\n').lower()
    if married == 'yes':
        userData["married"] = "Yes"
        break
    elif married == 'no':
        userData["married"] = "No"
        break  
    else:
        print(f"{married} is an incorrect entry.\nPlease type correct output")
        userData["married"]=married


while True:
    try:

        userData["dependencies"] = int(input('\nHow many people depend on you financially? (Type digits only) \n'))
        if type(userData["dependencies"]) == int: 
            break

    except ValueError:
        print("Input format is invalid. Type digits only")
        continue


while True:

    try:
        userData["annual_income"] = int(input('What is your annual income (before tax)?\n'))
        if type(userData["annual_income"]) == int:
            break

    except ValueError:
        print("Input format is invalid. Type digits only")
        continue

while True:
    try:
        userData["account_balance"]=int(input("\nAccount Balance: \n"))
        start_balance = userData["account_balance"]
        if type(userData["account_balance"]) == int:
            break

    except ValueError:
        print("Input format is invalid. Type digits only")


while True:
    try:

        last_credit_date=input("Last Pay Date (in DD/MM/YYYY): \n")
        userData["last_credit_date"]=date_convert((last_credit_date))
        break

    except ValueError:
        print("Oops!  Date format is wrong.  Try again in correct format.")

while True:

    try:
        next_pay_date=input("\nNext Pay Date (in DD/MM/YYYY): \n")
        userData["next_pay_date"]=date_convert((next_pay_date))
        break

    except ValueError:
        print("Oops!  Date format is wrong.  Try again in correct format.")

while True:
    try:
         if type(account_check(userData["account_balance"])) == int: 
                break

    except ValueError:
        print("Input format is invalid. Type digits only")




balance_over_time(userData["account_balance"])



user_copy = userData.copy()
masterData.append(user_copy)
print(user_copy)

In [ ]:
#functions

def age(birthdate):
    today = date.today()
    currage = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return currage

def account_check(balance):
    balance = int(balance)
    if balance >= (3*userData["annual_income"]):
        userData["AccountBalanceCheck"] = "OverBalance"
    else:
        userData["AccountBalanceCheck"] = "UnderBalance"
    return userData["AccountBalanceCheck"]
        

def date_convert(dates):
    daten=datetime.datetime.strptime(dates,"%d/%m/%Y").date()
    return daten
    
def balance_over_time(account_balance):
    monthbasis = list(reversed(range(22)))
    print("Current Account Balance is: ", account_balance)
    
    for i in monthbasis[::7]:
        #Account balance three weeks before next pay
        print(i)
        next_date = userData["next_pay_date"] - timedelta(days=i)
        print(next_date)
        account_balance = account_balance - 100
        userData["new_balance"] = account_balance
#     print("New account balance after one week is: ", account_balance)
    
    print("New account balance one week before next pay date is: GHS ", account_balance)


# With Loan Input

## Require parameters for agent 

What questions are we asking the agent

AGENT NAME:

ACCOUNT NAME: 

ACCOUNT NUMBER:

BUSINESS ADDRESS:
(PLEASE INDICATE LANDMARKS IF POSSIBLE)

EMAIL ADDRESS:

TELEPHONE NUMBER:

NATURE OF BUSINESS:

NO.OFOUTLETS:

OUTLET 1: OUTLET 2: OUTLET 3:
OUTLET 4: OUTLET 5:

MOBILE NUMBER:

BUSINESS REG NO:
(ENCLOSE A COPY OF CERTIFICATION)

In [ ]:
masterData = []
userData= {}

In [ ]:
# With Loan Input
# easier to do 3x calculation

print("Answer this questionnare\n")


userData["firstname"]=input("What is your agent name: ").lower()

userData["accountname"]=input("What is your account name: ").lower()

userData["business_add"]=input("What is your business address: ").lower()

userData["busi_nature"]=input("What is your nature of business: ").lower()

while True:
    try:
        userData["outlet"]=int(input("\nNo. of outlets: \n"))
        outlets = userData["outlet"]
        if type(userData["outlet"]) == int:
            break

    except ValueError:
        print("Input format is invalid. Type digits only")

# while True:
#     try:

#         birthday=input("What is your birthday? (in DD/MM/YYYY): ")

#         birthdate = date_convert((birthday))
#         birthdate=datetime.datetime.strptime(birthday,"%d/%m/%Y").date()
#         userData["age"]=age((birthdate))
#         break
#     except ValueError:
#         print("\n")
#         print("Oops!  Date format is wrong.  Try again in correct format.")


# while True:
#     married = input('\nAre you married?\n\nA) Yes \nB) No\n').lower()
#     if married == 'yes':
#         userData["married"] = "Yes"
#         break
#     elif married == 'no':
#         userData["married"] = "No"
#         break  
#     else:
#         print(f"{married} is an incorrect entry.\nPlease type correct output")
#         userData["married"]=married


# while True:
#     try:

#         userData["dependencies"] = int(input('\nHow many people depend on you financially? (Type digits only) \n'))
#         if type(userData["dependencies"]) == int: 
#             break

#     except ValueError:
#         print("Input format is invalid. Type digits only")
#         continue


# while True:

#     try:
#         userData["annual_income"] = int(input('What is your annual income (before tax)?\n'))
#         if type(userData["annual_income"]) == int:
#             break

#     except ValueError:
#         print("Input format is invalid. Type digits only")
#         continue

# while True:
#     try:
#         userData["account_balance"]=int(input("\nAccount Balance: \n"))
#         start_balance = userData["account_balance"]
#         if type(userData["account_balance"]) == int:
#             break

#     except ValueError:
#         print("Input format is invalid. Type digits only")


# while True:
#     try:

#         last_credit_date=input("Last Pay Date (in DD/MM/YYYY): \n")
#         userData["last_credit_date"]=date_convert((last_credit_date))
#         break

#     except ValueError:
#         print("Oops!  Date format is wrong.  Try again in correct format.")

# while True:

#     try:
#         next_pay_date=input("\nNext Pay Date (in DD/MM/YYYY): \n")
#         userData["next_pay_date"]=date_convert((next_pay_date))
#         break

#     except ValueError:
#         print("Oops!  Date format is wrong.  Try again in correct format.")

# while True:
#     try:
#          if type(account_check(userData["account_balance"])) == int: 
#                 break

#     except ValueError:
#         print("Input format is invalid. Type digits only")




# balance_over_time(userData["account_balance"])



# user_copy = userData.copy()
# masterData.append(user_copy)
# print(user_copy)

In [ ]:
# With Loan Input
# easier to do 3x calculation

print("Answer this questionnare\n")


userData["firstname"]=input("What is your first name: ").lower()
    
   

while True:
    married = input('\nAre you married?\n\nA) Yes \nB) No\n').lower()
    if married == 'yes':
        userData["married"] = "Yes"
        break
    elif married == 'no':
        userData["married"] = "No"
        break  
    else:
        print(f"{married} is an incorrect entry.\nPlease type correct output")
        userData["married"]=married


while True: 
    try:

        last_credit_date=input("Last Pay Date (in DD/MM/YYYY): \n")
        userData["last_credit_date"]=date_convert((last_credit_date))
        break

    except ValueError:
        print("Oops!  Date format is wrong.  Try again in correct format.")

            


print("hey")
balance_over_time(userData["account_balance"])



user_copy = userData.copy()
masterData.append(user_copy)
print(user_copy)

In [ ]:
#Convert Master Data from dict to pandas dataframe

df = pd.DataFrame.from_dict(masterData)
df

# Loan Eligibility Categorization

In [ ]:
inelligible = []
elligible = []

In [ ]:
for index in range(len(masterData)):
    if masterData[index]["new_balance"] <= 50 or masterData[index]["AccountBalanceCheck"] == 'UnderBalance':
        user_copy = masterData[index]
#         print(user_copy)
        inelligible.append(user_copy)
    else:
        elig_copy = masterData[index]
#         print(user_copy)
        elligible.append(elig_copy)
print("Inelligble customers", inelligible)
print('\n')
print("Elligble customers", elligible)

# Threshold

if account is below 50 one week before next pay, 

If userData["AccountBalanceCheck"]=='UnderBalance'


Not elliglbe for loan. pls re-check application and try again


else


if paycheck between x y z disburse a b c


# With Loan Range

In [ ]:
## With Loan Range


userData= {}

# print("Answer this questionnare\n")

userData["firstname"]=input("What is your first name: ").lower()

userData["lastname"]=input("\nWhat is your last name: ").lower()

birthday=input("What is your birthday? (in DD/MM/YYYY): ")
birthdate = date_convert((birthday))
birthdate=datetime.datetime.strptime(birthday,"%d/%m/%Y").date()
userData["age"]=age((birthdate))

while True:
    married = input('\nAre you married?\n\nA) Yes \nB) No\n').lower()
    if married == 'yes':
        userData["married"] = "Yes"
        break
    elif married == 'no':
        userData["married"] = "No"
        break  
    else:
        print(f"{married} is an incorrect entry.\nPlease type correct output")
        userData["married"]=married
        continue

userData["dependencies"] = int(input('\nHow many people depend on you financially? (Type number only) \n'))


while True:
    annual_income=input("\nSelect A, B, C or D\nWhat is your annual income(before tax)? \nA) < 1000Ghs \nB) 1000Ghs - 3000Ghs\nC) 3000Ghs - 4000Ghs \nD) > 4000Ghs\n").upper()
    if annual_income == 'A':
#         userData["annual_income"] = "< 1000Ghs"
        userData["income_bracket"] = "Tier 4"
    #calculating the average of income range selected
        userData["annual_income"] = 500
        break
    elif annual_income == 'B':
#         userData["annual_income"] = "1000Ghs - 3000Ghs"
        userData["income_bracket"] = "Tier 3"
     #calculating the average of income range selected
        userData["annual_income"] = 2000
 
        break
    if annual_income == 'C':
#         userData["annual_income"] = "3000Ghs - 4000Ghs"
        userData["income_bracket"] = "Tier 2"
     #calculating the average of income range selected
        userData["annual_income"] = 3500
 
        break
    elif annual_income == 'D':
#         userData["annual_income"] = "4000Ghs+"
        userData["income_bracket"] = "Tier 1"
     #calculating the average of income range selected
        userData["annual_income"] = 5000
 
        break
    else:
        print(f"{annual_income} is incorrect income.\nPlease select correct income range")
        userData["annual_income"]=annual_income
        continue

userData["account_balance"]=int(input("\nAccount Balance: \n"))
start_balance = userData["account_balance"]

last_credit_date=input("Last Pay Date (in DD/MM/YYYY): \n")
userData["last_credit_date"]=date_convert((last_credit_date))

next_pay_date=input("\nNext Pay Date (in DD/MM/YYYY): \n")
userData["next_pay_date"]=date_convert((next_pay_date))

account_check(userData["account_balance"])


balance_over_time(userData["account_balance"])



user_copy = userData.copy()
masterData.append(user_copy)


In [ ]:
#Convert Inelligible Data from dict to pandas dataframe

inelligible_df = pd.DataFrame.from_dict(inelligible)
inelligible_df


In [ ]:

#Convert Elligible Data from dict to pandas dataframe

elligible_df = pd.DataFrame.from_dict(elligible)
elligible_df